# Fine-tune with Pre-trained Models

In practice the dataset we use is relative small, so that we do not train an neural network from scratch, namely staring from random initialized parameters. Instead, it is common to train a neural network on a large-scale dataset and then use it either as an initialization or a fixed feature extractor. On [predict.ipynb](./predict.ipynb) we explained how to do the feature extraction, this tutorial will focus on how to use pre-trained model to fine tune a new network.

The idea of fine-tune is that, we take a pre-trained model, replace the last fully-connected layer with new one, which outputs the desired number of classes and initializes with random values. Then we train as normal except that we may often use a smaller learning rate since we may already very close the final result. 

We will use pre-trained models on the Imagenet dataset to fine-tune the smaller caltech-256 dataset as an example. But note that it can be used to other datasets as well, even for quite different applications such as face identification. 

We will show that, even with simple hyper-parameters setting, we can match and even outperform state-of-the-art results on caltech-256.

| Network | Accuracy | 
| --- | --- | 
| Resnet-50 | 77.4% | 
| Resnet-152 | 86.4% | 

## Prepare data

We follow the standard protocal to sample 60 images from each class as the training set, and the rest for the validation set. We resize images into 256x256 size and pack them into the rec file. The scripts to prepare the data is as following. 


```sh
wget http://www.vision.caltech.edu/Image_Datasets/Caltech256/256_ObjectCategories.tar
tar -xf 256_ObjectCategories.tar

mkdir -p caltech_256_train_60
for i in 256_ObjectCategories/*; do
    c=`basename $i`
    mkdir -p caltech_256_train_60/$c
    for j in `ls $i/*.jpg | shuf | head -n 60`; do
        mv $j caltech_256_train_60/$c/
    done
done

python ~/mxnet/tools/im2rec.py --list True --recursive True caltech-256-60-train caltech_256_train_60/
python ~/mxnet/tools/im2rec.py --list True --recursive True caltech-256-60-val 256_ObjectCategories/
python ~/mxnet/tools/im2rec.py --resize 256 --quality 90 --num-thread 16 caltech-256-60-val 256_ObjectCategories/
python ~/mxnet/tools/im2rec.py --resize 256 --quality 90 --num-thread 16 caltech-256-60-train caltech_256_train_60/
```

The following codes download the pre-generated rec files. It may take a few minutes.

In [1]:
#import os, urllib
#def download(url):
#    filename = url.split("/")[-1]
#    if not os.path.exists(filename):
#        urllib.urlretrieve(url, filename)
#download('http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec')
#download('http://data.mxnet.io/data/caltech-256/caltech-256-60-val.rec')

Next we define the function which returns the data iterators.

In [1]:
import mxnet as mx

def get_iterators(batch_size, data_shape=(3, 224, 224)):
    train = mx.io.ImageRecordIter(
        path_imgrec         = 'caltech-256-60-train.rec',
        data_name           = 'data',
        label_name          = 'softmax_label',
        batch_size          = batch_size,
        data_shape          = data_shape,
        shuffle             = True,
        rand_crop           = True,
        rand_mirror         = True)
    val = mx.io.ImageRecordIter(
        path_imgrec         = 'caltech-256-60-val.rec',
        data_name           = 'data',
        label_name          = 'softmax_label',
        batch_size          = batch_size,
        data_shape          = data_shape,
        rand_crop           = False,
        rand_mirror         = False)
    return (train, val)

In [1]:
import mxnet as mx
train_1 = mx.io.ImageRecordIter(
    path_imgrec         = 'caltech-256-60-train.rec',
    data_name           = 'data',
    label_name          = 'softmax_label',
    batch_size          = 100,
    data_shape          = (3,244,244),
    shuffle             = True,
    rand_crop           = True,
    rand_mirror         = True)

In [4]:
train_1.provide_label

[]

We then download a pretrained 50-layer ResNet model and load into memory. 

Note. If `load_checkpoint` reports error, we can remove the downloaded files and try `get_model` again.

In [7]:
#def get_model(prefix, epoch):
#    download(prefix+'-symbol.json')
#    download(prefix+'-%04d.params' % (epoch,))

# get_model('http://data.mxnet.io/models/imagenet/resnet/50-layers/resnet-50', 0)
sym, arg_params, aux_params = mx.model.load_checkpoint('resnet-50', 0)

## Train

We first define a function which replaces the the last fully-connected layer for a given network. 

In [3]:
def get_fine_tune_model(symbol, arg_params, num_classes, layer_name='flatten0'):
    """
    symbol: the pre-trained network symbol
    arg_params: the argument parameters of the pre-trained model
    num_classes: the number of classes for the fine-tune datasets
    layer_name: the layer name before the last fully-connected layer
    """
    all_layers = sym.get_internals()
    net = all_layers[layer_name+'_output']
    net = mx.symbol.FullyConnected(data=net, num_hidden=num_classes, name='fc1')
    net = mx.symbol.SoftmaxOutput(data=net, name='softmax')
    new_args = dict({k:arg_params[k] for k in arg_params if 'fc1' not in k})
    return (net, new_args)

Now we create a module. We first call `init_params` to randomly initialize parameters, next use `set_params` to replace all parameters except for the last fully-connected layer with pre-trained model. 

In [4]:
# cpu version
import logging
head = '%(asctime)-15s %(message)s'
logging.basicConfig(level=logging.DEBUG, format=head)

def fit(symbol, arg_params, aux_params, train, val, batch_size, num_cpus):
    devs = [mx.cpu(i) for i in range(num_cpus)]
    mod = mx.mod.Module(symbol=new_sym, context=devs)
    mod.bind(data_shapes=train.provide_data, label_shapes=train.provide_label)
    mod.init_params(initializer=mx.init.Xavier(rnd_type='gaussian', factor_type="in", magnitude=2))
    mod.set_params(new_args, aux_params)
    mod.fit(train, val, 
        num_epoch=8,
        batch_end_callback = mx.callback.Speedometer(batch_size, 10),
        kvstore='device',
        optimizer='sgd',
        optimizer_params={'learning_rate':0.01},
        eval_metric='acc')

Then we can start training. We use AWS EC2 g2.8xlarge, which has 8 GPUs.

In [5]:
num_classes = 256
batch_per_gpu = 16
num_cpus = 4

(new_sym, new_args) = get_fine_tune_model(sym, arg_params, num_classes)

batch_size = batch_per_gpu * num_cpus
(train, val) = get_iterators(batch_size)

# out of memory
fit(new_sym, new_args, aux_params, train, val, batch_size, num_cpus)

AssertionError: 


As can be seen, even for 8 data epochs, we can get 78% validation accuracy. It matches the state-of-the-art results training on caltech-256 alone, e.g. [VGG](http://www.robots.ox.ac.uk/~vgg/research/deep_eval/). 

Next we try to use another pretrained model. It uses the complete Imagenet dataset, which is 10x larger than the Imagenet 1K classes one, and is trained with a 3x deeper Resnet network. 